In [1]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import math

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print('using device', device)

importing Jupyter notebook from CustomDataset.ipynb
importing Jupyter notebook from Model.ipynb
using device cpu


In [2]:
torch.cuda.empty_cache()

In [3]:
dataset = ControlsDataset()
#dataset.labels.set_type("angular")
dataset.labels.set_type("categorical")
dataset.images.set_grayscale(True)
print("Data size", len(dataset.labels))
dataset.make_dataloaders()

Data size 11019
Total training stacks 35
Total validation stacks 9


In [4]:
net = ConvNet(color_channels = 1, outputs = 21, dataset = dataset).to(device)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

units after conv 512
conv parameters:  164576
fc parameters:  166933
number of parameters:  331509


In [5]:
epochs = 1
optimizer = optim.Adam(net.parameters(), lr=0.0001)

counts = dataset.labels.dataframe.groupby('Category')['ID'].count()
weights = max(counts)/counts
weights = torch.Tensor(weights)

criterion = nn.CrossEntropyLoss(weight=weights) # Changed from Mean-Squared to Cross-Entropy
#criterion = nn.MSELoss()
net.report_period = 1
net.fit(device,epochs,optimizer,criterion)

tensor([10, 10,  3, 10, 19,  5, 16, 10, 20, 10,  5, 10, 19, 20, 10, 14, 15, 15,
        10, 20, 20, 15, 19, 19,  5, 10, 10,  3, 15,  3,  2, 10,  3,  3, 20, 19,
        15, 10, 20, 15, 10, 15, 20, 10,  3, 19,  5,  4, 10, 20, 20, 20, 15, 10,
        10, 10, 10, 19, 15, 19,  2,  2,  5, 19, 10, 15, 10, 10, 10, 10, 17, 20,
        15, 15, 15,  3,  3, 20, 10,  3, 20,  5, 20,  3, 14, 20, 10, 20, 19, 10,
        15, 15,  3,  3, 15,  4, 20,  5,  3, 10,  3, 15, 15, 15, 10,  5, 15, 19,
        20, 20, 20,  3, 20,  3, 15,  5, 15, 10, 15, 10, 10,  3, 15,  5, 17, 15,
        20, 17, 10, 10,  5, 10,  2, 20, 20, 15, 15, 19, 19, 10,  5, 15, 20,  3,
        10,  3,  5, 10, 10, 20, 20, 10, 15, 15, 15, 10, 10,  3, 10,  5,  3,  5,
        10, 10,  5,  5, 10, 10, 10, 10, 20, 20, 10, 19, 20, 20, 20, 15, 10, 20,
        20,  2, 15, 20, 10, 20,  3,  5, 20,  2,  5, 10,  4, 19, 20,  5,  5,  5,
         5, 15, 10, 15, 15, 19, 15, 10, 15, 10,  5, 10,  5,  5, 10,  5, 10,  5,
        10, 15, 15, 19,  2, 20, 10,  3, 

0,7	Loss:3.02926	Allocated:0.0GB	Cached:0.0GB

tensor([ 5, 14, 10,  5, 10,  3,  5, 10, 17, 12, 10, 10, 10,  3, 10, 10, 14, 15,
        14,  5, 10, 20, 20, 20, 10, 10,  5,  5, 17, 10,  5,  4, 15, 20,  3,  5,
         5, 14, 15, 20,  5,  5, 17, 20, 17, 10,  5, 10, 17, 17, 20, 10,  2, 10,
         5, 10,  5, 10, 10, 10, 20, 10,  5, 10, 20,  3, 10, 15, 15, 10, 16, 10,
        14, 15, 10, 10,  3,  3, 10,  2,  5, 10, 10, 15, 20,  2, 20, 10,  5, 20,
        11,  5, 17, 10, 15, 20, 10, 17, 15, 20, 20, 15, 20, 10, 10, 10, 10,  5,
        20, 20, 20, 10, 10,  5, 10, 17,  5, 10,  5, 15, 20, 10, 15, 20,  5, 10,
         5,  2, 16,  5,  3, 10, 17, 10, 15,  5,  2,  5,  5, 10,  5,  5, 20, 10,
         2, 10,  3, 20, 14, 20, 10, 15, 10,  5, 14,  3, 10, 10, 10,  2,  5, 15,
        10,  5, 10, 10, 10, 10, 15, 10,  5, 10,  2, 20,  5, 15,  5,  5,  3, 20,
         5,  9, 10, 17,  2,  5, 10,  5,  5, 10, 20, 15,  3, 10, 10,  2, 15, 10,
         3,  5, 10, 10, 15,  5, 10, 20,  5,  3,  5,  5,  5, 10, 14,  5,  

0,14	Loss:3.04673	Allocated:0.0GB	Cached:0.0GB

tensor([ 3,  5,  5, 10,  5, 20, 14,  5, 10,  5, 20,  5, 10,  5, 20, 10, 10,  5,
         5, 10, 10, 20,  3,  5, 10, 10,  5, 10, 20, 10, 10,  5, 10,  5,  5,  2,
        17, 10, 10,  5,  9, 10,  9,  5, 10, 20, 17, 10,  5,  5, 15,  3,  5,  5,
         5, 14, 10, 15,  5, 15, 10,  5, 10, 20,  5, 17, 17, 10, 10,  3,  5, 10,
         5, 17,  9, 10,  5, 10,  5, 10, 10, 10, 15,  5,  3, 15, 15, 10, 17,  5,
         5,  5,  5, 10, 20, 10, 15,  5, 10, 15, 20, 20,  5, 20, 20, 20,  5, 10,
        10,  5,  3,  5, 17, 10,  5, 20,  5,  3, 10, 20,  3,  5,  9,  3, 10,  3,
        10, 20, 17, 20,  5,  5, 20,  5, 10, 10, 10, 10, 10,  3, 10,  5,  5, 15,
        14, 10, 10, 15, 10, 10,  5, 10,  5,  5, 10, 10, 17, 10, 10,  5,  5, 10,
        20, 17, 17,  5, 17, 10,  5, 17, 10, 10,  5, 17, 10,  5, 10, 15, 15,  5,
        20, 10,  5,  5, 10,  3, 10, 20, 10,  5,  5, 10, 10, 15,  5,  5,  5, 15,
        10, 14,  3,  5,  3,  5, 10,  5, 10, 17, 10, 10,  3, 17,  5,  5, 

KeyboardInterrupt: 

In [ ]:
# End of training Validation Test
net.load("snapshots/0.727_model.pt")
total, correct = net.score(device,dataset, "categorical")
print("{:.1f}% classified correctly".format(100*correct/total))

In [ ]:
# Beginning of training Validation Test
net.load("snapshots/3.002_model.pt")
total, correct = net.score(device,dataset, "categorical")
print("{:.1f}% classified correctly".format(100*correct/total))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#Additional Info when using cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))